# Возврат исходных документов

В этом разделе показано как доработать приложение, созданное в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart), чтобы оно возвращало документы, использованные для генерации.

Такое поведение поможет пользователям ознакомиться с источниками данных.

## Установка зависимостей

Для разработки используются модели генерации и эмбеддингов GigaChat, а также векторное хранилище FAISS.
Вы можете использовать любую [векторное хранилище](/docs/modules/data_connection/vectorstores/) или [ретривер](/docs/modules/data_connection/retrievers/)

Установите пакеты с помощью команды:

In [ ]:
%pip install --upgrade --quiet  gigachain langchainhub faiss-cpu bs4

## Цепочка без возврата исходных документов

Пример цепочки, которая не возвращает исходные документы, разобранный в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart):

In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
# Загрузка, разделение на части и индексация содержимого блога.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents=splits, embedding=GigaChatEmbeddings(credentials="<автризационные_данные>", verify_ssl_certs=False))

# Извлечение данных и генерация с помощью релевантных фрагментов блога.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = GigaChat(credentials="<автризационные_данные>", verify_ssl_certs=False)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complicated task into smaller and simpler steps. This allows an agent to better understand and plan ahead for the task. CoT (Chain of Thought) and Tree of Thoughts are techniques that help enhance model performance on complex tasks by decomposing them into manageable steps. Task decomposition can be done using LLMs with simple prompting, task-specific instructions, or with human inputs.'

## Добавление исходных документов

LCEL позволяет возвращать документы, полученные ретривером:

In [6]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("What is Task Decomposition")

{'context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous task which generat